# Interactive Heat Source Visualization

This notebook provides interactive Plotly visualizations to understand:
1. How heat diffuses from sources over time (animation)
2. What sensors "see" - temperature readings over time
3. The inverse problem - finding sources from sensor data
4. Loss landscape visualization

In [1]:
import sys
import os
sys.path.insert(0, os.path.join(os.getcwd(), '..', 'src'))
sys.path.insert(0, os.path.join(os.getcwd(), '..', 'data', 'Heat_Signature_zero-starter_notebook'))

import numpy as np
import pickle
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

from simulator import Heat2D
from optimizer import HeatSourceOptimizer

print("Imports complete!")

Imports complete!


## 1. Animated Heat Diffusion

Watch how heat spreads from point sources over time.

In [2]:
# Setup simulation parameters
Lx, Ly = 2.0, 1.0
nx, ny = 100, 50
kappa = 0.01
dt = 0.01
nt = 200
bc = 'dirichlet'
T0 = 0.0

# Create two heat sources
sources = [
    {'x': 0.5, 'y': 0.5, 'q': 1.5},
    {'x': 1.5, 'y': 0.5, 'q': 1.0},
]

# Place some sensors
sensors_xy = np.array([
    [0.3, 0.3],
    [0.7, 0.7],
    [1.0, 0.5],
    [1.3, 0.3],
    [1.7, 0.7],
])

print(f"Sources: {sources}")
print(f"Sensors: {sensors_xy.shape[0]} locations")

Sources: [{'x': 0.5, 'y': 0.5, 'q': 1.5}, {'x': 1.5, 'y': 0.5, 'q': 1.0}]
Sensors: 5 locations


In [3]:
# Run simulation
solver = Heat2D(Lx, Ly, nx, ny, kappa, bc=bc)
times, Us = solver.solve(dt=dt, nt=nt, T0=T0, sources=sources, store_every=5)

# Sample at sensors
Y = np.array([solver.sample_sensors(U, sensors_xy) for U in Us])

print(f"Simulation complete: {len(times)} frames")
print(f"Temperature range: {Us.min():.2f} to {Us.max():.2f}")

Simulation complete: 41 frames
Temperature range: 0.00 to 33.29


In [4]:
# Create animated heatmap
x = np.linspace(0, Lx, nx)
y = np.linspace(0, Ly, ny)

# Build frames for animation
frames = []
for i, (t, U) in enumerate(zip(times, Us)):
    frames.append(go.Frame(
        data=[go.Heatmap(
            z=U.T,
            x=x,
            y=y,
            colorscale='Hot',
            zmin=0,
            zmax=Us.max(),
            colorbar=dict(title='Temp')
        )],
        name=str(i)
    ))

# Initial frame
fig = go.Figure(
    data=[go.Heatmap(
        z=Us[0].T,
        x=x,
        y=y,
        colorscale='Hot',
        zmin=0,
        zmax=Us.max(),
        colorbar=dict(title='Temperature')
    )],
    frames=frames
)

# Add source markers
fig.add_trace(go.Scatter(
    x=[s['x'] for s in sources],
    y=[s['y'] for s in sources],
    mode='markers',
    marker=dict(size=15, color='cyan', symbol='star', line=dict(width=2, color='white')),
    name='Heat Sources'
))

# Add sensor markers
fig.add_trace(go.Scatter(
    x=sensors_xy[:, 0],
    y=sensors_xy[:, 1],
    mode='markers',
    marker=dict(size=12, color='lime', symbol='circle', line=dict(width=2, color='white')),
    name='Sensors'
))

# Animation controls
fig.update_layout(
    title='Heat Diffusion Animation (★ = Sources, ● = Sensors)',
    xaxis_title='x',
    yaxis_title='y',
    height=500,
    updatemenus=[dict(
        type='buttons',
        showactive=False,
        y=1.15,
        x=0.5,
        xanchor='center',
        buttons=[
            dict(label='▶ Play',
                 method='animate',
                 args=[None, dict(frame=dict(duration=50, redraw=True),
                                  fromcurrent=True,
                                  mode='immediate')]),
            dict(label='⏸ Pause',
                 method='animate',
                 args=[[None], dict(frame=dict(duration=0, redraw=False),
                                    mode='immediate')])
        ]
    )],
    sliders=[dict(
        active=0,
        yanchor='top',
        xanchor='left',
        currentvalue=dict(prefix='Time: ', visible=True, xanchor='center'),
        steps=[dict(args=[[f.name], dict(mode='immediate', frame=dict(duration=0, redraw=True))],
                    label=f'{times[i]:.2f}s',
                    method='animate') for i, f in enumerate(frames)]
    )]
)

fig.show()

## 2. Sensor Readings Over Time

This is what we observe - just the temperature at sensor locations.

In [5]:
# Plot sensor readings
fig = go.Figure()

colors = px.colors.qualitative.Set1
for i in range(sensors_xy.shape[0]):
    fig.add_trace(go.Scatter(
        x=times,
        y=Y[:, i],
        mode='lines',
        name=f'Sensor {i+1} @ ({sensors_xy[i,0]:.1f}, {sensors_xy[i,1]:.1f})',
        line=dict(color=colors[i % len(colors)], width=2)
    ))

fig.update_layout(
    title='Sensor Temperature Readings Over Time',
    xaxis_title='Time (s)',
    yaxis_title='Temperature',
    height=400,
    hovermode='x unified'
)

fig.show()

## 3. The Inverse Problem

Given ONLY the sensor readings (above), can we find where the sources are?

Let's visualize the loss landscape for a single source.

In [6]:
# For simplicity, use a single source case
single_source = [{'x': 1.0, 'y': 0.5, 'q': 1.0}]
solver_single = Heat2D(Lx, Ly, nx, ny, kappa, bc=bc)
times_s, Us_s = solver_single.solve(dt=dt, nt=100, T0=T0, sources=single_source)
Y_observed = np.array([solver_single.sample_sensors(U, sensors_xy) for U in Us_s])

# Add noise (like in real test data)
noise_level = 0.05
Y_noisy = Y_observed + noise_level * np.random.randn(*Y_observed.shape) * Y_observed.max()

print(f"True source: x=1.0, y=0.5, q=1.0")
print(f"Observed data shape: {Y_noisy.shape}")

True source: x=1.0, y=0.5, q=1.0
Observed data shape: (101, 5)


In [7]:
# Compute loss landscape for varying x and y (fixed q=1.0)
x_range = np.linspace(0.2, 1.8, 25)
y_range = np.linspace(0.1, 0.9, 20)

loss_grid = np.zeros((len(x_range), len(y_range)))

print("Computing loss landscape...")
for i, x_test in enumerate(x_range):
    for j, y_test in enumerate(y_range):
        test_sources = [{'x': x_test, 'y': y_test, 'q': 1.0}]
        _, Us_test = solver_single.solve(dt=dt, nt=100, T0=T0, sources=test_sources)
        Y_pred = np.array([solver_single.sample_sensors(U, sensors_xy) for U in Us_test])
        loss_grid[i, j] = np.sqrt(np.mean((Y_pred - Y_noisy) ** 2))
    if (i + 1) % 5 == 0:
        print(f"  {i+1}/{len(x_range)} rows complete")

print(f"Loss range: {loss_grid.min():.4f} to {loss_grid.max():.4f}")

Computing loss landscape...
  5/25 rows complete
  10/25 rows complete
  15/25 rows complete
  20/25 rows complete
  25/25 rows complete
Loss range: 0.8169 to 7.4287


In [8]:
# Interactive 3D loss surface
fig = go.Figure(data=[go.Surface(
    x=x_range,
    y=y_range,
    z=loss_grid.T,
    colorscale='Viridis',
    colorbar=dict(title='RMSE')
)])

# Mark true source location
fig.add_trace(go.Scatter3d(
    x=[1.0], y=[0.5], z=[loss_grid.min() - 0.1],
    mode='markers',
    marker=dict(size=10, color='red', symbol='diamond'),
    name='True Source'
))

fig.update_layout(
    title='Loss Landscape: RMSE vs Source Position (q fixed at 1.0)',
    scene=dict(
        xaxis_title='Source X',
        yaxis_title='Source Y',
        zaxis_title='RMSE',
        camera=dict(eye=dict(x=1.5, y=1.5, z=0.8))
    ),
    height=600
)

fig.show()

In [9]:
# 2D contour view
fig = go.Figure(data=go.Contour(
    x=x_range,
    y=y_range,
    z=loss_grid.T,
    colorscale='Viridis',
    contours=dict(showlabels=True),
    colorbar=dict(title='RMSE')
))

# Mark true source
fig.add_trace(go.Scatter(
    x=[1.0], y=[0.5],
    mode='markers',
    marker=dict(size=15, color='red', symbol='star'),
    name='True Source'
))

# Mark sensors
fig.add_trace(go.Scatter(
    x=sensors_xy[:, 0],
    y=sensors_xy[:, 1],
    mode='markers',
    marker=dict(size=10, color='lime', symbol='circle'),
    name='Sensors'
))

fig.update_layout(
    title='Loss Contours: Darker = Lower RMSE = Better Fit',
    xaxis_title='Source X',
    yaxis_title='Source Y',
    height=500
)

fig.show()

## 4. Real Test Data Exploration

Let's look at some actual test samples.

In [10]:
# Load test data
with open(os.path.join(os.getcwd(), '..', 'data', 'heat-signature-zero-test-data.pkl'), 'rb') as f:
    test_data = pickle.load(f)

meta = test_data['meta']
samples = test_data['samples']

print(f"Test dataset: {len(samples)} samples")
print(f"Meta: dt={meta['dt']}, q_range={meta['q_range']}")

Test dataset: 80 samples
Meta: dt=0.004, q_range=(0.5, 2)


In [11]:
# Interactive sample explorer
def plot_sample(sample_idx):
    sample = samples[sample_idx]
    
    Y = sample['Y_noisy']
    sensors = np.array(sample['sensors_xy'])
    n_sources = sample['n_sources']
    sample_meta = sample['sample_metadata']
    
    times = np.arange(Y.shape[0]) * meta['dt']
    
    # Create subplots
    fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=('Sensor Readings', 'Sensor Locations'),
        specs=[[{"type": "scatter"}, {"type": "scatter"}]]
    )
    
    # Left: Sensor readings
    colors = px.colors.qualitative.Set1
    for i in range(Y.shape[1]):
        fig.add_trace(go.Scatter(
            x=times, y=Y[:, i],
            mode='lines',
            name=f'S{i+1}',
            line=dict(color=colors[i % len(colors)])
        ), row=1, col=1)
    
    # Right: Domain with sensors
    # Draw domain boundary
    fig.add_trace(go.Scatter(
        x=[0, 2, 2, 0, 0],
        y=[0, 0, 1, 1, 0],
        mode='lines',
        line=dict(color='gray', dash='dash'),
        name='Domain',
        showlegend=False
    ), row=1, col=2)
    
    # Sensors
    fig.add_trace(go.Scatter(
        x=sensors[:, 0],
        y=sensors[:, 1],
        mode='markers+text',
        marker=dict(size=15, color='blue'),
        text=[f'S{i+1}' for i in range(len(sensors))],
        textposition='top center',
        name='Sensors'
    ), row=1, col=2)
    
    fig.update_layout(
        title=f"Sample: {sample['sample_id']} | Sources: {n_sources} | BC: {sample_meta['bc']} | κ={sample_meta['kappa']}",
        height=400,
        showlegend=True
    )
    
    fig.update_xaxes(title_text='Time (s)', row=1, col=1)
    fig.update_yaxes(title_text='Temperature', row=1, col=1)
    fig.update_xaxes(title_text='x', range=[-0.1, 2.1], row=1, col=2)
    fig.update_yaxes(title_text='y', range=[-0.1, 1.1], row=1, col=2)
    
    return fig

# Show first sample
fig = plot_sample(0)
fig.show()

In [12]:
# Create dropdown to explore all samples
from ipywidgets import interact, IntSlider

def show_sample(idx):
    fig = plot_sample(idx)
    fig.show()

interact(show_sample, idx=IntSlider(min=0, max=len(samples)-1, step=1, value=0, description='Sample:'))

interactive(children=(IntSlider(value=0, description='Sample:', max=79), Output()), _dom_classes=('widget-inte…

<function __main__.show_sample(idx)>

## 5. Optimization Visualization

Watch the optimizer search for source locations.

In [13]:
# Pick a sample and run optimization with history tracking
sample = samples[0]
print(f"Optimizing: {sample['sample_id']}")

optimizer = HeatSourceOptimizer(Lx, Ly, nx, ny)
est_sources, rmse = optimizer.estimate_sources(
    sample, meta,
    q_range=meta['q_range'],
    method='L-BFGS-B',
    n_restarts=1,
    max_iter=50,
    track_history=True
)

print(f"Estimated sources: {est_sources}")
print(f"Final RMSE: {rmse:.6f}")
print(f"History: {len(optimizer.history)} evaluations")

Optimizing: sample_0
Estimated sources: [(np.float64(1.1773509113174476), np.float64(0.5088959313652617), np.float64(1.206207599776342))]
Final RMSE: 0.100255
History: 128 evaluations


In [14]:
# Plot optimization path
history = optimizer.history
n_sources = sample['n_sources']

# Extract positions over iterations
iterations = list(range(len(history)))
rmse_values = [h['rmse'] for h in history]
x_values = [[h['params'][i*3] for h in history] for i in range(n_sources)]
y_values = [[h['params'][i*3+1] for h in history] for i in range(n_sources)]
q_values = [[h['params'][i*3+2] for h in history] for i in range(n_sources)]

# RMSE convergence
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('RMSE Convergence', 'Source X Position', 'Source Y Position', 'Source Intensity q')
)

fig.add_trace(go.Scatter(x=iterations, y=rmse_values, mode='lines', name='RMSE'), row=1, col=1)

for i in range(n_sources):
    fig.add_trace(go.Scatter(x=iterations, y=x_values[i], mode='lines', name=f'Source {i+1}'), row=1, col=2)
    fig.add_trace(go.Scatter(x=iterations, y=y_values[i], mode='lines', name=f'Source {i+1}', showlegend=False), row=2, col=1)
    fig.add_trace(go.Scatter(x=iterations, y=q_values[i], mode='lines', name=f'Source {i+1}', showlegend=False), row=2, col=2)

fig.update_layout(height=600, title='Optimization Progress')
fig.show()

In [15]:
# Animated optimization path on 2D domain with colored trails
sensors = np.array(sample['sensors_xy'])

# Get true sources if available
true_sources = sample.get('true_sources', None)

# Colors for each source
source_colors = ['red', 'orange', 'magenta', 'cyan', 'yellow']

# Build frames - each frame updates trails + current positions for all sources
frames = []
history_subset = history[::5]  # Every 5th iteration

for frame_idx in range(len(history_subset)):
    frame_data = []
    
    for src_idx in range(n_sources):
        color = source_colors[src_idx % len(source_colors)]
        
        # Trail: all positions up to this frame
        trail_x = [history_subset[i]['params'][src_idx*3] for i in range(frame_idx + 1)]
        trail_y = [history_subset[i]['params'][src_idx*3 + 1] for i in range(frame_idx + 1)]
        
        # Trail trace
        frame_data.append(go.Scatter(
            x=trail_x, y=trail_y,
            mode='lines+markers',
            line=dict(color=color, width=2, dash='dot'),
            marker=dict(size=4, color=color, opacity=0.5),
            name=f'Source {src_idx+1} Trail'
        ))
        
        # Current position trace
        curr_x = history_subset[frame_idx]['params'][src_idx*3]
        curr_y = history_subset[frame_idx]['params'][src_idx*3 + 1]
        frame_data.append(go.Scatter(
            x=[curr_x], y=[curr_y],
            mode='markers',
            marker=dict(size=20, color=color, symbol='star', line=dict(width=2, color='white')),
            name=f'Source {src_idx+1}'
        ))
    
    # Trace indices: 0=domain, 1=sensors, 2=true sources, then 2 traces per source (trail + position)
    trace_indices = list(range(3, 3 + n_sources * 2))
    
    frames.append(go.Frame(
        data=frame_data,
        traces=trace_indices,
        name=str(frame_idx)
    ))

# Initial figure data
initial_data = [
    # Domain boundary (trace 0)
    go.Scatter(x=[0, 2, 2, 0, 0], y=[0, 0, 1, 1, 0],
               mode='lines', line=dict(color='gray', dash='dash'), name='Domain'),
    # Sensors (trace 1)
    go.Scatter(x=sensors[:, 0], y=sensors[:, 1],
               mode='markers', marker=dict(size=12, color='blue'), name='Sensors'),
    # True sources (trace 2) - green diamonds
    go.Scatter(
        x=[s['x'] for s in true_sources] if true_sources else [],
        y=[s['y'] for s in true_sources] if true_sources else [],
        mode='markers',
        marker=dict(size=18, color='lime', symbol='diamond', line=dict(width=2, color='darkgreen')),
        name='True Sources'
    ),
]

# Add initial trail + position traces for each source
for src_idx in range(n_sources):
    color = source_colors[src_idx % len(source_colors)]
    init_x = history_subset[0]['params'][src_idx*3]
    init_y = history_subset[0]['params'][src_idx*3 + 1]
    
    # Trail trace (starts with single point)
    initial_data.append(go.Scatter(
        x=[init_x], y=[init_y],
        mode='lines+markers',
        line=dict(color=color, width=2, dash='dot'),
        marker=dict(size=4, color=color, opacity=0.5),
        name=f'Source {src_idx+1} Trail'
    ))
    
    # Current position
    initial_data.append(go.Scatter(
        x=[init_x], y=[init_y],
        mode='markers',
        marker=dict(size=20, color=color, symbol='star', line=dict(width=2, color='white')),
        name=f'Source {src_idx+1}'
    ))

fig = go.Figure(data=initial_data, frames=frames)

fig.update_layout(
    title='Optimizer Convergence: ★ Estimated → ◆ True Sources',
    xaxis=dict(range=[-0.1, 2.1], title='x'),
    yaxis=dict(range=[-0.1, 1.1], title='y'),
    height=500,
    updatemenus=[dict(
        type='buttons',
        showactive=False,
        y=1.15,
        x=0.5,
        buttons=[
            dict(label='▶ Play',
                 method='animate',
                 args=[None, dict(frame=dict(duration=100, redraw=True), mode='immediate')]),
            dict(label='⏸ Pause',
                 method='animate',
                 args=[[None], dict(frame=dict(duration=0, redraw=False), mode='immediate')])
        ]
    )],
    sliders=[dict(
        active=0,
        yanchor='top',
        xanchor='left',
        currentvalue=dict(prefix='Iteration: ', visible=True, xanchor='center'),
        steps=[dict(
            args=[[str(i)], dict(mode='immediate', frame=dict(duration=0, redraw=True))],
            label=str(i * 5),
            method='animate'
        ) for i in range(len(frames))]
    )]
)

fig.show()

## 6. Observed vs Predicted Comparison

How well does our estimate match the observations?

In [16]:
# Simulate with estimated sources
sample_meta = sample['sample_metadata']
est_source_dicts = [{'x': s[0], 'y': s[1], 'q': s[2]} for s in est_sources]

solver_est = Heat2D(Lx, Ly, nx, ny, sample_meta['kappa'], bc=sample_meta['bc'])
_, Us_est = solver_est.solve(
    dt=meta['dt'],
    nt=sample_meta['nt'],
    T0=sample_meta['T0'],
    sources=est_source_dicts
)
Y_pred = np.array([solver_est.sample_sensors(U, sensors) for U in Us_est])
Y_obs = sample['Y_noisy']

times = np.arange(len(Y_obs)) * meta['dt']

In [17]:
# Compare observed vs predicted
n_sensors = Y_obs.shape[1]
fig = make_subplots(rows=n_sensors, cols=1,
                    subplot_titles=[f'Sensor {i+1}' for i in range(n_sensors)],
                    shared_xaxes=True)

for i in range(n_sensors):
    # Observed
    fig.add_trace(go.Scatter(
        x=times, y=Y_obs[:, i],
        mode='lines',
        name=f'Observed {i+1}',
        line=dict(color='blue', width=2)
    ), row=i+1, col=1)
    
    # Predicted
    fig.add_trace(go.Scatter(
        x=times, y=Y_pred[:, i],
        mode='lines',
        name=f'Predicted {i+1}',
        line=dict(color='red', width=2, dash='dash')
    ), row=i+1, col=1)

fig.update_layout(
    title=f'Observed (blue) vs Predicted (red dashed) | RMSE = {rmse:.4f}',
    height=150 * n_sensors + 100,
    showlegend=True
)
fig.update_xaxes(title_text='Time (s)', row=n_sensors, col=1)

fig.show()

## Summary

Key insights from these visualizations:

1. **Heat diffuses smoothly** from point sources over time
2. **Sensors only see local temperatures** - the inverse problem is under-determined
3. **Loss landscape has a clear minimum** near the true source location
4. **Optimizer converges** by iteratively improving source estimates
5. **Predicted readings match observed** when estimation is good